In [1]:
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from models.CNN import create_cnn_model

In [2]:
def load(f):
    # Move up one directory when loading the data
    file_path = os.path.join('../../../', f)
    return np.load(file_path)['arr_0']

# Load the data
x_train = load('kmnist-train-imgs.npz')/ 255.0
x_test = load('kmnist-test-imgs.npz')/ 255.0
y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')
# Reshape the data for CNN input
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [3]:
input_shape = x_train.shape[1:]  # 784 for KMNIST
num_classes = y_train.max() + 1
x_train_aug, x_val, y_train_aug, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [4]:
hyperparameters = {
    'num_layers': 3,
    'filters': [32, 64, 64],
    'kernel_sizes': [(3, 3), (3, 3), (3, 3)],
    'activations': ['relu', 'relu', 'relu'],
    'dense_units': 64,
    'dense_activation': 'relu',
    'pooling': 'avg',
    'batch_normalization': True
}

In [6]:
def cutout(image, mask_size=16):
    """Applies Cutout augmentation to an image."""
    h, w, _ = image.shape
    x = np.random.randint(0, w)
    y = np.random.randint(0, h)

    # Ensure the cutout does not go out of bounds
    x1 = np.clip(x - mask_size // 2, 0, w)
    x2 = np.clip(x + mask_size // 2, 0, w)
    y1 = np.clip(y - mask_size // 2, 0, h)
    y2 = np.clip(y + mask_size // 2, 0, h)

    # Create a copy of the image to avoid modifying the original
    image_copy = image.copy()
    image_copy[y1:y2, x1:x2, :] = 0  # Set the cutout region to black

    return image_copy

In [7]:
best_model_mask = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_no_mask = create_cnn_model(input_shape, num_classes, hyperparameters)
datagen_mask = ImageDataGenerator(
    preprocessing_function = lambda x: cutout(x, mask_size=16) # Zoom in on the image
)
datagen_no_mask = ImageDataGenerator()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
datagen_val = ImageDataGenerator()
# Wrap the datagen with our custom generator
train_generator_mask = datagen_mask.flow(x_train_aug, y_train_aug, batch_size=32)
train_generator_no_mask = datagen_no_mask.flow(x_train_aug, y_train_aug, batch_size=32)
val_generator = datagen_val.flow(x_val, y_val, batch_size=32)
# Train the model with custom augmentation
history_mask = best_model_mask.fit(train_generator_mask,
                        epochs=20,
                        validation_data=val_generator,  
                        verbose=1,
                        callbacks=[early_stopping])
history_no_mask = best_model_no_mask.fit(train_generator_no_mask,
                        epochs=20,
                        validation_data=val_generator,
                        verbose=1,
                        callbacks=[early_stopping])

# Evaluate on the test set
test_loss_mask, test_accuracy_mask = best_model_mask.evaluate(x_test, y_test, verbose=0)
test_loss_no_mask, test_accuracy_no_mask = best_model_no_mask.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy with Mask Injection: {test_accuracy_mask:.4f}")
print(f"Test loss with Mask Injection: {test_loss_mask:.4f}")
print(f"Test accuracy without Mask Injection: {test_accuracy_no_mask:.4f}")
print(f"Test loss without Mask Injection: {test_loss_no_mask:.4f}")

Epoch 1/20
1688/1688 [==============================] - 16s 6ms/step - loss: 0.4519 - accuracy: 0.8605 - val_loss: 0.0979 - val_accuracy: 0.9708
Epoch 2/20
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2385 - accuracy: 0.9232 - val_loss: 0.0929 - val_accuracy: 0.9715
Epoch 3/20
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1929 - accuracy: 0.9388 - val_loss: 0.0865 - val_accuracy: 0.9735
Epoch 4/20
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1733 - accuracy: 0.9447 - val_loss: 0.0838 - val_accuracy: 0.9742
Epoch 5/20
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1555 - accuracy: 0.9495 - val_loss: 0.0480 - val_accuracy: 0.9850
Epoch 6/20
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1418 - accuracy: 0.9539 - val_loss: 0.0420 - val_accuracy: 0.9867
Epoch 7/20
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1336 - accuracy: 0.9560 - val_loss: 0.0547 - val_ac